# Fine-tuning BERT model for Binary Text Classification

## Preparation for Google Collab

In [58]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [59]:
import os
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/MyBert'

In [61]:
os.listdir("/content/drive/MyDrive/Colab Notebooks")

['CH05a_BERT_fine-tuning.ipynb', 'MyBert']

In [62]:
if os.getcwd() != "/content/drive/MyDrive/Colab Notebooks/MyBert":
    os.chdir("/content/drive/MyDrive/Colab Notebooks/MyBert")

In [63]:
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/MyBert'

In [64]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/MyBert")

[]

In [65]:
!apt-get install tree && tree -a "/content/drive/MyDrive/Colab Notebooks/MyBert"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
/content/drive/MyDrive/Colab Notebooks/MyBert

0 directories, 0 files


## Loading pre-trained model and tokenizer

In [66]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [67]:
model = DistilBertForSequenceClassification.from_pretrained(
    model_path, id2label={0: "NEG", 1: "POS"}, label2id={"NEG": 0, "POS": 1}
)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Loading popular IMDB dataset

In [68]:
from datasets import load_dataset

# to take entire dataset from original train 25 K AND TEST 25K
"""
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

"""
# to take smaller portion 4K for train, 1K for test and 1K for validation
imdb_train = load_dataset("imdb", split="train[:2000]+train[-2000:]")
imdb_test = load_dataset("imdb", split="test[:500]+test[-500:]")
imdb_val = load_dataset("imdb", split="test[500:1000]+test[-1000:-500]")

In [69]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

((4000, 2), (1000, 2), (1000, 2))

In [70]:
imdb_train[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [71]:
enc_train = imdb_train.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_test = imdb_test.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_val = imdb_val.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [72]:
enc_train.shape, enc_test.shape, enc_val.shape

((4000, 4), (1000, 4), (1000, 4))

In [73]:
enc_train[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [74]:
import pandas as pd

pd.DataFrame(enc_train)

,text,label,input_ids,attention_mask
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[101, 1045, 12524, 1045, 2572, 8025, 1011, 375...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[101, 1000, 1045, 2572, 8025, 1024, 3756, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,If only to avoid making this type of film in t...,0,"[101, 2065, 2069, 2000, 4468, 2437, 2023, 2828...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,This film was probably inspired by Godard's Ma...,0,"[101, 2023, 2143, 2001, 2763, 4427, 2011, 2643...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"[101, 2821, 1010, 2567, 1012, 1012, 1012, 2044...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
3995,A hit at the time but now better categorised a...,1,"[101, 1037, 2718, 2012, 1996, 2051, 2021, 2085...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3996,I love this movie like no other. Another time ...,1,"[101, 1045, 2293, 2023, 3185, 2066, 2053, 2060...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3997,This film and it's sequel Barry Mckenzie holds...,1,"[101, 2023, 2143, 1998, 2009, 1005, 1055, 8297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3998,'The Adventures Of Barry McKenzie' started lif...,1,"[101, 1005, 1996, 7357, 1997, 6287, 18506, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Preparing training settings with TrainingArguments and Trainer class

TrainingArguments is the subset of the arguments we use in our example scripts which relate to the training loop itself.

In [75]:
from transformers import TrainingArguments, Trainer
from torch import cuda

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./MyIMDBModel",
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy="steps",
    # TensorBoard log directory
    logging_dir="./logs",
    logging_steps=50,
    # other options : no, steps
    evaluation_strategy="steps",
    save_strategy="epoch",
    fp16=cuda.is_available(),
    report_to=[],
)
training_args

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=50,
eval_strategy=IntervalStrategy.STEPS,
eval_use

Let's design our evaluation metrics as follows:

In [76]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1": f1, "Precision": precision, "Recall": recall}

In [77]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
    # training arguments that we defined above
    args=training_args,
    # training and validation dataset
    train_dataset=enc_train,
    eval_dataset=enc_val,
    compute_metrics=compute_metrics,
)
trainer

In [78]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.607900,0.385553,0.852000,0.851429,0.857497,0.852000
100,0.339300,0.433839,0.828000,0.824950,0.852573,0.828000
150,0.278200,0.305051,0.885000,0.884723,0.888733,0.885000
200,0.199600,0.293084,0.897000,0.896710,0.901511,0.897000
250,0.208700,0.316819,0.886000,0.885531,0.892430,0.886000
300,0.096400,0.366476,0.888000,0.887899,0.889402,0.888000
350,0.105900,0.336165,0.900000,0.899998,0.900026,0.900000


* For small portion

We have 4K training examples and the batch size is 32, which means each epoch takes 4k/32 = 125 steps. For 3 epoc, this will have 375 steps as wee see in the output.

* For full dataset

We have 25K training  examples with the batch size pf 32, so we will have 25K/32 ~=782 (roughly) steps. for 3 epoch we will have 2346 steps to run as seen in the progress bar

Trainer model keeps the best model at the end. Lets evaluate the model on the train/test/validation

In [79]:
q = [trainer.evaluate(eval_dataset=data) for data in [enc_train, enc_val, enc_test]]
pd.DataFrame(q, index=["train", "val", "test"]).iloc[:, :5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.047899,0.989,0.989000,0.989018,0.989
val,0.335838,0.895,0.894999,0.895014,0.895
test,0.252059,0.919,0.918982,0.919377,0.919


In [80]:
# saving the best fine-tuned model & tokenizer
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

('MyBestIMDBModel/tokenizer_config.json',
 'MyBestIMDBModel/special_tokens_map.json',
 'MyBestIMDBModel/vocab.txt',
 'MyBestIMDBModel/added_tokens.json',
 'MyBestIMDBModel/tokenizer.json')

In [81]:
!tree -a "/content/drive/MyDrive/Colab Notebooks/MyBert"

/content/drive/MyDrive/Colab Notebooks/MyBert
├── MyBestIMDBModel
│   ├── config.json
│   ├── model.safetensors
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   ├── tokenizer.json
│   ├── training_args.bin
│   └── vocab.txt
└── MyIMDBModel
    ├── checkpoint-125
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    ├── checkpoint-250
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    └── checkpoint-375
        ├── config.json
        ├── model.safetensors
        ├── optimizer.pt
        ├── rng_state.pth
        ├── scheduler.pt
        ├── trainer_state.json
        └── training_args.bin

5 directories, 28 files


In [ ]:
def get_prediction(text):
    inputs = tokenizer(
        text, padding=True, truncation=True, max_length=250, return_tensors="pt"
    ).to(device)
    outputs = model(inputs["input_ids"].to(device), inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

In [ ]:
model.to(device)
text = "I didn't like the movie since it bored me "
get_prediction(text)[1].item()

Use the model with pipeline

In [ ]:
from transformers import (
    pipeline,
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
)

model = DistilBertForSequenceClassification.from_pretrained("MyBestIMDBModel")
tokenizer = DistilBertTokenizerFast.from_pretrained("MyBestIMDBModel")
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
nlp("the movie was very impressive")

In [ ]:
nlp("the script of the picture was very poor")